# How to read data from BigQuery

This notebook demonstrates two ways to use BigQuery with R:
1. by using SQL
2. by using only R code (dplyr) to extract the data of interest from BigQuery

## Setup

In [ ]:
lapply(c('skimr'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

In [ ]:
# Load the libraries into memory
library(bigrquery)
library(dplyr)
library(skimr)
library(ggplot2)

In [ ]:
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

## Retrieve filtered data.
Let’s retrieve a subset of fields and samples in metadata for [1000 Genomes](http://www.internationalgenome.org/data "1000 Genomes").

We can do this in two different ways and they both return the exact same results

1. using SQL and bigrquery
2. using dplyr and dbplyr

### Option 1: Retrieve filtered data from BigQuery using bigrquery.
To learn more about SQL syntax see the [BigQuery standard SQL reference](https://cloud.google.com/bigquery/docs/reference/standard-sql/).

In [ ]:
# Run a query 
phase1_samples_tbl <- bigrquery::bq_project_query(
    BILLING_PROJECT_ID,
    query = '
SELECT
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
FROM
  `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
WHERE
  -- Only include information for samples in phase 1.
  In_Phase1_Integrated_Variant_Set = TRUE
  -- Only include information for samples with these relationships.
  AND Relationship IN ("", "unrel", "child", "father", "mother",
  "mat grandmother", "pat grandmother",
  "mat grandfather", "pat grandfather")
')

phase1_samples <- bigrquery::bq_table_download(phase1_samples_tbl)

In [ ]:
dim(phase1_samples)

In [ ]:
print(skim(phase1_samples))

In [ ]:
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

### Option 2: Retrieve filtered data from BigQuery using dplyr.
To learn more about dplyr see [R for Data Science Chapter 5 Data transformation](http://r4ds.had.co.nz/transform.html "Chapter 5").

In [ ]:
# Create a "connection" to a public BigQuery dataset.
dbcon <- bigrquery::src_bigquery(project = 'bigquery-public-data',
                                 dataset = 'human_genome_variants',
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table.
sample_info <- dplyr::tbl(dbcon, '1000_genomes_sample_info')

In [ ]:
phase1_only <- sample_info %>% filter(
    In_Phase1_Integrated_Variant_Set == TRUE,
    Relationship %in% c(
      '', 'unrel', 'child', 'father', 'mother',
      'mat grandmother', 'pat grandmother',
      'mat grandfather', 'pat grandfather')
  )
# Nothing is computed on BigQuery yet.

In [ ]:
sample_fields <- phase1_only %>% select(
  Sample,
  Gender,
  Relationship,
  Population,
  Population_Description,
  Super_Population,
  Super_Population_Description,
  Total_Exome_Sequence,
  Main_Project_E_Platform,
  Main_Project_E_Centers
  )
# Still nothing computed on BigQuery.

In [ ]:
# Optional: take a look at the SQL.
dplyr::show_query(sample_fields)

In [ ]:
# Optional: See how much data this will return.
sample_fields %>% summarize(cnt = n()) %>% collect()

In [ ]:
# Execute the query and return all results into an in-memory table in R.
phase1_samples <- sample_fields %>% collect()

In [ ]:
dim(phase1_samples)

In [ ]:
ggplot(data = phase1_samples,
       aes(x=Relationship, fill=Relationship)) +
  geom_bar() +
  theme(axis.text.x = element_text(angle = 50, hjust = 1, vjust = 1))

# Provenance

In [ ]:
devtools::session_info()

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.